Дан набор данных из 20 чистых и 20 грязных тарелок.

Необходимо обучить классификатор определять грязные тарелки.

Результат - классифицировать 400 тестовых тарелок.

In [ ]:
import numpy as np 
import pandas as pd

import os
print(os.listdir("../input"))

In [ ]:
import zipfile
with zipfile.ZipFile('/kaggle/input/platesv2/plates.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/kaggle/working/')
    
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))

In [ ]:
data_root = '/kaggle/working/plates/'
print(os.listdir(data_root))

In [ ]:
#создание 2х новых папок train и val для разделения данных на тренировочные и валидационные. Каждую 6ю фотографию переложим в папку val
import shutil 
from tqdm import tqdm

train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 3 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [ ]:
!ls train

In [ ]:
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy

In [ ]:
#аугментация разными способами, вырезаем часть исходного изображения после чего делаем его размером 224, зеркалим и т.д... 
# тем самым искуственно увеличиваем количество тренировочных данных для более сильного обучения

from torchvision import transforms, models

train_transforms_1 = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  
])

train_transforms_2 = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_transforms_3 = transforms.Compose([
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_transforms_4 = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_transforms_5 = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_transforms_6 = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.RandomRotation(degrees=(1,359)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_transforms_7 = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ColorJitter(0.6, 0.6, 0.3, 0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_transforms_8 = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.RandomGrayscale(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset_1 = torchvision.datasets.ImageFolder(train_dir, train_transforms_1)
train_dataset_2 = torchvision.datasets.ImageFolder(train_dir, train_transforms_2)
train_dataset_3 = torchvision.datasets.ImageFolder(train_dir, train_transforms_3)
train_dataset_4 = torchvision.datasets.ImageFolder(train_dir, train_transforms_4)
train_dataset_5 = torchvision.datasets.ImageFolder(train_dir, train_transforms_5)
train_dataset_6 = torchvision.datasets.ImageFolder(train_dir, train_transforms_6)
train_dataset_7 = torchvision.datasets.ImageFolder(train_dir, train_transforms_7)
train_dataset_8 = torchvision.datasets.ImageFolder(train_dir, train_transforms_8)

train_dataset = torch.utils.data.ConcatDataset([train_dataset_1, train_dataset_2, train_dataset_3, train_dataset_4, train_dataset_5, train_dataset_6, train_dataset_7, train_dataset_8])

val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

# ImageFolder при передаче каждого изображения привяжет к нему метку в какой папке он его взял cleaned/dirty (родительской папки)
# Чтобы ImageFolder передавал изображения как тензоры, а не как изображения открытые специальной библиотекой PIL, необходимо указывать 2й параметр "train/val_transforms", т.е сделать трансформации.
# 1я трансформация - ужать до 224*224, 2я превратить изображение в тензор, 3я отнормировать изображения до тех параметнров, на изображениях с которыми ResNet был обучен изначально.

batch_size = 18    # размер батча 8 картинок. батчи должны быть полными, т.е. кол-во элементов (в нашем случае 40) должно нацело делиться на кол-во элементов в батче
train_dataloader = torch.utils.data.DataLoader(      # train_dataloader объект который отдает батчи с изображениями
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [ ]:
len(train_dataloader), len(train_dataset)

In [ ]:
len(val_dataloader), len(val_dataset)

In [ ]:
# отобразим, что у нас в dataloader, при этом проделаем все трансформации в обратном порядке 
X_batch, y_batch = next(iter(train_dataloader))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
plt.imshow(X_batch[0].permute(1, 2, 0).numpy() * std + mean);

In [ ]:
# функция, для отображения что у нас получается в трейновом dataloader
def show_input(input_tensor, title=''):
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)

X_batch, y_batch = next(iter(train_dataloader))

for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

In [ ]:
# тренировка Нейронной сети
# 1. модель resnet18 (архитектура), 2. лосс ф-я nn.CrossEntropyLoss, 3. метод градиентного спуска Adam, 4. планировщик как будет уменьшаться шаг градиентного спуска
model = models.resnet152(pretrained=True) #(include_top=False, classes=2, input_shape = (224,224,3))

# изначально обученую сеть с 152ю слоями resnet152 мы "заморозим", т.е. не будем изменять
for param in model.parameters():
    param.requires_grad = False

# поменяем последний полносвязный слой, т.к. оригинальный resnet18 в конце разбивает изображения на 1000 классов, а у нас их 2 - чистые/грязные
# model.fc.in_features - кол-во нейронов на вход, 2- кол-во нейронов на выход  - 2 класса
model.fc = torch.nn.Linear(model.fc.in_features, 2)

# перекладываем нашу сеть на gpu для ускорения процесса обработки данных
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# т.к. у нас 2 класса подойдет бинарная кросс-энтропия
loss = torch.nn.CrossEntropyLoss()

# метод градиентного спуска Adam
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)   #optim.Adam(model.parameters(), lr=1.0e-3)

# с течением обучения нужно уменьшать шаг градиентного спуска
# Decay LR by a factor of 0.1 every 7 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# функция train_model каждую эпоху проходит по батчам. 
# если это обучение, то она делает степы (бэкворд+град спуск), если валидация - просто считаем качество, чтобы наблюдать не начали ли мы переобучаться

def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:  # фаза трейна 
            if phase == 'train':
                dataloader = train_dataloader # dataloader дает батчи из трейнового датасета, т.е. из папки трейн
                scheduler.step() # планировщик делает шаг, который говорит что произошла одна эпоха
                model.train()  # перевод модели в фазу трейна. даже если мы не делаем бэкворд, даже если мы не считаем градиент и не изменяем веса, тем не менее в формард пассе у нас могут меняться параметры слоя батч-нормализации
            else:
                dataloader = val_dataloader
                model.eval()   # перевод модели в фазу валидации. это важно (!), т.к. во время валидации НС может изменяться

            running_loss = 0.
            running_acc = 0.

            # итерации по dataloader, он отдает батчи с одним тензором изображения и с одним тензором лейбла
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad() # обнуляем градиент, чтобы он не накапливался

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'): # обертка, которая активирует те градиенты, которые "не заморожены", это сильно ускоряет расчет
                    preds = model(inputs) # inputs передаем в модель и считаем прогнозы, выходы это не вероятности, а некоторые активации нейронов
                    loss_value = loss(preds, labels) # расчет лосс ф-ции
                    preds_class = preds.argmax(dim=1) # прогноз класса,к которому относится наше изображение. берем argmax - нейрон с максимальной активацией

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # расчет статистик. нужен для понимания уменьшается ли loss с течением эпох или вышел на платто
                # нам нужны loss-ы и accuracy, т.к. єто метрика нашего соревнования и нужно понимать как она изменяется с течением эпох
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader) # усредняем, деля на кол-во батчей в эпохе, т.е. на len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model

In [ ]:
# функция, которая получает на вход все аргументы (model, loss, optimizer, scheduler) + количество эпох (сколько мы хотим обычать нашу модель)
train_model(model, loss, optimizer, scheduler, num_epochs=100);

In [ ]:
# переносим все тестовые изображения в папку test/unknown, т.к. ImageFolder при передаче каждого изображения привязывает лейбл(название корневой папки, ранее клин/дерти) и если не создать новую папку в папке, то ImageFolder-у негде будет брать лейбл
# теперь у тестовых изображений будет лейбл unknown. неважно, как он будет называться, важно чтобы он был.
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

In [ ]:
# мы не знаем какие названия/id-ишники генерируются у изображений, когда мы просим у DataLoader дать следующий батч.
# поэтому нужно переписать ImageFolder, чтобы он отдавал не просто tuple с изображением и меткой, а еще чтобы давал имя или путь к изображению.
# эту задачу решает __getitem__
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms) # теперь test_dataset отдает tuple с 3мя значениями

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
test_dataset

In [ ]:
# прогоним все изображения через сеть и получим прогнозы

# переходим в модель eval, т.е. никаких изменений внутри происходить не будет, жестко зафиксированы все веса и параметры НС
model.eval()

# наполняем 2 списка - наши предсказания и пути к изображениям
test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy()) #перегон прогнозов (подаются на вход) в вероятности первого класса (от 0 до 1)
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

In [ ]:
# вывод изображений тарелок с вероятностью что они "грязные"
inputs, labels, paths = next(iter(test_dataloader))

for img, pred in zip(inputs, test_predictions):
    show_input(img, title=pred)